In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import LabelEncoder

DIR = '../data/imdb/'

FEATURES_PATH = DIR + 'matrix.bin'
NAMES_PATH = DIR + 'names.txt'

# Load Files
FILE_LIMIT = 44500
FEATURES = np.fromfile(FEATURES_PATH, np.float32).reshape(-1, 128)[:FILE_LIMIT]
with open(NAMES_PATH, encoding='utf-8') as names_file:
    NAMES = names_file.read().split("\n")[:FILE_LIMIT]

label_encoder = LabelEncoder()
LABELS = label_encoder.fit_transform(NAMES)

# Bound By Number of Samples
MIN_SAMPLES = 40
MAX_SAMPLES = 45
unique, counts = np.unique(LABELS, return_counts=True)
COUNTS = counts[LABELS]
SELECTION = np.logical_and(COUNTS > MIN_SAMPLES, COUNTS < MAX_SAMPLES)

FEATURES = FEATURES[SELECTION]
LABELS = LABELS[SELECTION]
N_PERSONS = len(set(LABELS))

print(N_PERSONS, len(FEATURES) / N_PERSONS)

11 42.90909090909091


In [70]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics


classifier = KNeighborsClassifier(20)

# Cross Validation Score
scores = cross_val_score(classifier, FEATURES, LABELS, cv=3)
print("Cross Validation Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


FEATURES_TRAIN, FEATURES_TEST, LABELS_TRAIN, LABELS_TEST = train_test_split(FEATURES, LABELS, test_size=0.2)
classifier.fit(FEATURES_TRAIN, LABELS_TRAIN)

correct = 0.0
total = 0.0

for distances, indices, truth in zip(*classifier.kneighbors(FEATURES_TEST), LABELS_TEST):
    labels = LABELS_TRAIN[indices]
    label = np.bincount(labels).argmax()
    confidence = np.mean(labels == label)
    distance = np.mean(distances[labels == label])
    
#     print(label, confidence, distance)
    
    if confidence > 0.8:
        total += 1
        correct += label == truth
        

print(correct / total)

Cross Validation Accuracy: 0.69 (+/- 0.03)
0.9444444444444444
